In [165]:
import pandas as pd
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.snippets import sequence_padding
import numpy as np
import tensorflow.compat.v1 as tf
#import tensorflow as tf

In [164]:
tf.nn

<module 'tensorflow._api.v1.nn' from '/home/zyp/anaconda3/envs/simbert/lib/python3.6/site-packages/tensorflow/_api/v1/nn/__init__.py'>

In [11]:
dict_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/vocab.txt'

token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)

In [21]:
config_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/bert_model.ckpt'
bert = build_transformer_model(
    config_path,
    checkpoint_path,
    with_pool='linear',
    application='unilm',
    keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字表    
    return_keras_model=False,
)
encoder = keras.models.Model(bert.model.inputs, bert.model.outputs[0])

In [14]:
maxlen = 32

In [4]:
rexcel = pd.read_excel("./datasets/爱善大学堂百问百答模块.xlsx",sheet_name='问答明细')
rexcel["Unnamed: 1"][2:]
asks = rexcel["Unnamed: 1"][2:].apply(lambda x:tokenizer.encode(x, max_length=maxlen)[0]).tolist()
asks = sequence_padding(asks)

In [195]:
len(rexcel["Unnamed: 1"][2:][61])

19

In [197]:
with open("asks.txt","w") as f:
    f.write("\n".join(rexcel["Unnamed: 1"][2:].apply(lambda x:x.replace("\n","")).tolist()))

In [159]:
input_ids = tf.placeholder(dtype=tf.int32,shape=[None])

In [167]:
input_ids

<tf.Tensor 'Placeholder_204:0' shape=(?,) dtype=int32>

In [166]:
asks_lookup = tf.nn.embedding_lookup(asks2tensor,input_ids)

array([345, 340, 343, 346, 339, 159, 357, 129, 356,   9,  10, 358, 355,
       394,  11, 317, 314,  69, 328, 316], dtype=int32)

In [178]:
result = sess.run(asks_lookup,feed_dict={input_ids:tk.indices})

In [182]:
str(result[1],encoding="utf-8")

'怎么给年轻女顾客连带俏妃送给妈妈？'

In [33]:
texts = rexcel["Unnamed: 1"][2:].tolist()

In [23]:
asks_vecs = encoder.predict([asks,np.zeros_like(asks)],verbose=True)


400/400 [==============================] - 8s 19ms/step


In [24]:
asks_vecs = asks_vecs / (asks_vecs**2).sum(axis=1, keepdims=True)**0.5

In [30]:
vecs = asks_vecs.reshape(-1, 768)

In [34]:
def most_similar(text, topn=10):
    """检索最相近的topn个句子
    """
    token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
    vec = encoder.predict([[token_ids], [segment_ids]])[0]
    vec /= (vec**2).sum()**0.5
    sims = np.dot(vecs, vec)
    return [(texts[i], sims[i]) for i in sims.argsort()[::-1][:topn]]

In [105]:
def get_vector(text):
    token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
    vec = encoder.predict([[token_ids], [segment_ids]])[0]    
    return vec

In [201]:
most_similar("怎么跟客户介绍俏妃")

[('如何引导俏妃的好评反馈？', 0.8197763),
 ('怎么给年轻女顾客连带俏妃送给妈妈？', 0.753228),
 ('俏妃发快递时如何给客户发信息？', 0.75146115),
 ('如何对俏妃客户进行半个月后的连带回访？', 0.7250136),
 ('怎么给老顾客连带俏妃青春版送女儿？', 0.69393027),
 ('客户说：俏妃卫生巾有点贵？', 0.66068596),
 ('俏妃经典款（老款）的核心卖点是什么？', 0.6295761),
 ('客户说：我用棉柔卫生巾、网面卫生巾挺好的，不需要俏妃？', 0.5946022),
 ('俏妃青春版（少女）的核心卖点是什么？', 0.5549495),
 ('俏妃青春版如何做到超强防漏？', 0.554675)]

In [199]:
t_in = get_vector("怎么跟客户介绍俏妃") 

In [125]:
t_in /= (t_in **2).sum() ** 0.5

In [126]:
len(t_in)

768

In [127]:
t_in = tf.constant(t_in  )

In [54]:
ten1 = tf.get_tensor_by_name("Const:0")

AttributeError: module 'tensorflow.compat.v1' has no attribute 'get_tensor_by_name'

In [57]:
sess = tf.Session()

In [62]:
ten1 = sess.graph.get_tensor_by_name("Const_1:0")

In [63]:
ten2 = sess.graph.get_tensor_by_name("Const:0")

In [128]:
ten3 = tf.tensordot(ten1 , t_in,axes = 1)

In [173]:
result_dot = sess.run(ten3)

In [174]:
top_k = tf.math.top_k(result_dot,k=20)

In [175]:
tk = sess.run(top_k)

In [176]:
tk

TopKV2(values=array([0.8197763 , 0.75322807, 0.7514611 , 0.72501355, 0.69393027,
       0.6606859 , 0.629576  , 0.59460217, 0.5549495 , 0.554675  ,
       0.5545728 , 0.53730106, 0.52202106, 0.5108089 , 0.4986623 ,
       0.48109767, 0.4736092 , 0.45551768, 0.45472336, 0.44915724],
      dtype=float32), indices=array([345, 340, 343, 346, 339, 159, 357, 129, 356,   9,  10, 358, 355,
       394,  11, 317, 314,  69, 328, 316], dtype=int32))

In [177]:
tk.indices

array([345, 340, 343, 346, 339, 159, 357, 129, 356,   9,  10, 358, 355,
       394,  11, 317, 314,  69, 328, 316], dtype=int32)

In [136]:
rexcel["Unnamed: 1"].loc[rexcel["Unnamed: 1"] == "如何引导俏妃的好评反馈？"]

347    如何引导俏妃的好评反馈？
Name: Unnamed: 1, dtype: object

In [155]:
asks2tensor = tf.constant(rexcel["Unnamed: 1"][2:])

In [144]:
str_constant = sess.run(str_constant)

In [183]:
str(str_constant,encoding="utf-8")

'如何引导俏妃的好评反馈？'

In [157]:
asks2tensor.eval(session=sess)

array([b'\xe5\xad\xa9\xe5\xad\x909\xe5\xb2\x81\xef\xbc\x8c\xe6\x9c\x89\xe7\x82\xb9\xe8\x99\xab\xe7\x89\x99\xef\xbc\x8c\xe6\x80\x8e\xe4\xb9\x88\xe5\x8a\x9e\xef\xbc\x9f\xe9\x9c\x80\xe8\xa6\x81\xe6\x8b\x94\xe7\x89\x99\xe5\x90\x97\xef\xbc\x9f',
       b'\xe6\x9c\x89\xe5\x93\xaa\xe4\xba\x9b\xe8\xae\xa9\xe4\xbd\xa0\xe9\x95\xbf\xe5\xa4\xa7\xe5\x90\x8e\xe7\xaa\x81\xe7\x84\xb6\xe9\x86\x92\xe6\x82\x9f\xe7\x9a\x84\xe5\xb0\x8f\xe9\x81\x93\xe7\x90\x86\xef\xbc\x9f\n\n',
       b'\xe8\x89\xbe\xe7\x81\xb8\xe5\x88\x86\xe5\xad\xa3\xe8\x8a\x82\xe5\x90\x97\xef\xbc\x9f',
       b'\xe5\xa4\x8f\xe5\xa4\xa9\xe8\x89\xbe\xe7\x81\xb8\xe4\xbb\x80\xe4\xb9\x88\xe6\x97\xb6\xe9\x97\xb4\xe6\x95\x88\xe6\x9e\x9c\xe6\x9c\x80\xe4\xbd\xb3\xef\xbc\x9f\xe6\x99\x9a\xe4\xb8\x8a\xe5\x8f\xaf\xe4\xbb\xa5\xe5\x90\x97\xef\xbc\x9f\xe7\x81\xb8\xe5\xae\x8c\xe8\xb5\xb7\xe6\xb3\xa1\xe6\x80\x8e\xe4\xb9\x88\xe5\xa4\x84\xe7\x90\x86\xef\xbc\x9f\xe8\xaf\xb7\xe6\x95\x99\xe4\xb8\x80\xe4\xb8\x8b\xf0\x9f\x98\x8a ',
       b'\xe5\x85\xa8\xe6\x96\

In [184]:
type(tk)

tensorflow.python.ops.gen_nn_ops.TopKV2